In [1]:
import copy
import datetime
import pandas as pd
import os
from tqdm import tqdm, trange
import pymssql
from optparse import OptionParser, Option, OptionValueError

In [2]:
def get_data(database, command):
    
    db_setting = {'server' : '10.60.0.39', 
                  'user' : 'WarrantWeb',
                  'password' : 'WarrantWeb',
                  'database' : database,}
                  #'charset' : 'GBK'}

    conn = pymssql.connect(**db_setting)
    cursor = conn.cursor()
    data = pd.read_sql(command, conn)
    conn.close()

    return data

### Sample Join Code

In [153]:
# command = """ SELECT * FROM
#               (SELECT * FROM [ETFData].[dbo].[ETFNAV] WHERE 日期 = '2022-05-30') AS A
#               LEFT JOIN (SELECT [日期],[股票代號],[整合帳號]
#               FROM [ETFData].[dbo].[ETFPosition] WHERE 日期 = '2022-05-30') AS B
#               ON A.日期 = B.日期 and A.股票代號 = B.股票代號 """
# get_data('ETFData', command)

In [10]:
command = """ SELECT A.*, B.收盤價 FROM
              (SELECT * FROM [ETFData].[dbo].[ETFDetail] WHERE 股票代號 = '0056') AS A
              LEFT JOIN (SELECT [日期],[股票代號],[收盤價]
              FROM [ETFPL].[dbo].[StkPrice]) AS B
              ON A.日期 = B.日期 and A.標的代號 = B.股票代號
              where A.日期 = '2023-02-17' """
df = get_data('ETFData', command)
df = df.fillna(15436*200)
df['數值'] = df['數值'].astype(float).astype(int)

c:\Users\0012077\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


### 0056

In [11]:
df['市值'] = df['數值'] * df['收盤價']# * (df['權重(%)'] / 100)
df['股票名稱'] = df['股票名稱'] .str.encode('latin').str.decode('cp950')
df['標的名稱'] = df['標的名稱'] .str.encode('latin').str.decode('cp950')
df['商品種類'] = df['商品種類'] .str.encode('latin').str.decode('cp950')
df['單位'] = df['單位'] .str.encode('latin').str.decode('cp950')
df.head(20)

,日期,股票代號,股票名稱,標的代號,標的名稱,商品種類,數值,單位,金額,金額幣別,幣別,權重(%),契約年月,最後交易日,最後結算日,收盤價,市值
0,2023-02-17,0056,元大高股息,1102,亞泥,股票,105986839,ªÑ,,,,2.43,,1900-01-01,1900-01-01,42.85,4.541536e+09
1,2023-02-17,0056,元大高股息,1301,台塑,股票,38897284,ªÑ,,,,1.88,,1900-01-01,1900-01-01,90.10,3.504645e+09
2,2023-02-17,0056,元大高股息,1303,南亞,股票,46107849,ªÑ,,,,1.87,,1900-01-01,1900-01-01,75.70,3.490364e+09
3,2023-02-17,0056,元大高股息,1402,遠東新,股票,125434725,ªÑ,,,,2.15,,1900-01-01,1900-01-01,32.00,4.013911e+09
4,2023-02-17,0056,元大高股息,1802,台玻,股票,125827284,ªÑ,,,,1.57,,1900-01-01,1900-01-01,23.30,2.931776e+09
5,2023-02-17,0056,元大高股息,2006,東和鋼鐵,股票,33686114,ªÑ,,,,1.02,,1900-01-01,1900-01-01,56.70,1.910003e+09
6,2023-02-17,0056,元大高股息,202303TX,台指期貨,期貨,1109,¤f,,,,1.83,202303,2023-03-15,2023-03-15,3087200.00,3.423705e+09
7,2023-02-17,0056,元大高股息,2027,大成鋼,股票,83258965,ªÑ,,,,1.91,,1900-01-01,1900-01-01,42.80,3.563484e+09
8,2023-02-17,0056,元大高股息,2301,光寶科,股票,69802228,ªÑ,,,,2.62,,1900-01-01,1900-01-01,70.10,4.893136e+09
9,2023-02-17,0056,元大高股息,2303,聯電,股票,102216276,ªÑ,,,,2.74,,1900-01-01,1900-01-01,50.00,5.110814e+09


In [175]:
# df['股票名稱'].str.encode('utf-8').str.decode('big5')

In [156]:
df['市值'].sum() / 6870534000

27.18659078405696

In [157]:
3423704800 / 1109 / 200

15436.0

In [158]:
df['市值'].sum() #- 3087200.00 * 1109

186786396325.95

In [159]:
print(186891557787 - (183362691526 + 3087200.00 * 1109))
print((186891557787 - 186786396326) / 186786396326)

105161461.0
0.000563003853966221


### 00878

In [177]:
command = """ SELECT A.*, B.收盤價 FROM
              (SELECT * FROM [ETFData].[dbo].[ETFDetail] WHERE 股票代號 = '00878') AS A
              LEFT JOIN (SELECT [日期],[股票代號],[收盤價]
              FROM [ETFPL].[dbo].[StkPrice]) AS B
              ON A.日期 = B.日期 and A.標的代號 = B.股票代號
              where A.日期 = '2023-02-17' """
df2 = get_data('ETFData', command)
df2.loc[int(df2[df2.標的代號.apply(lambda x : 'TX' in x)].index.values), '收盤價'] = 15436*200
df2['數值'] = df2['數值'].astype(float).astype(int)

c:\Users\0012077\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [182]:
df2.tail(10)

,日期,股票代號,股票名稱,標的代號,標的名稱,商品種類,數值,單位,金額,金額幣別,幣別,權重(%),契約年月,最後交易日,最後結算日,收盤價,市值
24,2023-02-17,00878,°ê®õ¥ÃÄò°ªªÑ®§,2890,¥ÃÂ×ª÷,ªÑ²¼,274549460,ªÑ,,,,2.92,,1900-01-01,1900-01-01,17.40,4.777161e+09
25,2023-02-17,00878,°ê®õ¥ÃÄò°ªªÑ®§,2891,¤¤«Hª÷,ªÑ²¼,211087000,ªÑ,,,,2.94,,1900-01-01,1900-01-01,22.75,4.802229e+09
26,2023-02-17,00878,°ê®õ¥ÃÄò°ªªÑ®§,2892,²Ä¤@ª÷,ªÑ²¼,152619777,ªÑ,,,,2.47,,1900-01-01,1900-01-01,26.50,4.044424e+09
27,2023-02-17,00878,°ê®õ¥ÃÄò°ªªÑ®§,3045,¥xÆW¤j,ªÑ²¼,45384000,ªÑ,,,,2.68,,1900-01-01,1900-01-01,96.60,4.384094e+09
28,2023-02-17,00878,°ê®õ¥ÃÄò°ªªÑ®§,3702,¤jÁp¤j,ªÑ²¼,130449000,ªÑ,,,,3.93,,1900-01-01,1900-01-01,49.30,6.431136e+09
29,2023-02-17,00878,°ê®õ¥ÃÄò°ªªÑ®§,4904,»·¶Ç,ªÑ²¼,69709000,ªÑ,,,,2.94,,1900-01-01,1900-01-01,69.00,4.809921e+09
30,2023-02-17,00878,°ê®õ¥ÃÄò°ªªÑ®§,5876,¤W®ü°Ó»È,ªÑ²¼,83227590,ªÑ,,,,2.40,,1900-01-01,1900-01-01,47.15,3.924181e+09
31,2023-02-17,00878,°ê®õ¥ÃÄò°ªªÑ®§,C_NTD,CASH,²{ª÷,-2147483648,¤¸,5575039078.0000,NTD,NTD,0.00,,1900-01-01,1900-01-01,NaN,NaN
32,2023-02-17,00878,°ê®õ¥ÃÄò°ªªÑ®§,M_NTD,MARGIN,«OÃÒª÷,1325235725,¤¸,1325235725.0000,NTD,NTD,0.00,,1900-01-01,1900-01-01,NaN,NaN
33,2023-02-17,00878,°ê®õ¥ÃÄò°ªªÑ®§,PFUR_NTD,PAYABLES FOR UNITS REDEEMED,À³¥I¨ü¯qÅv³æ¦ì¶R¦^´Ú,-118510488,¤¸,-118510488.0000,NTD,NTD,0.00,,1900-01-01,1900-01-01,NaN,NaN


In [194]:
df2['市值'] = df2['數值'] * df2['收盤價']
# df2['市值'] = df2['市值'].fillna(df2['金額'])
df2['市值'] = df2['市值'].fillna(0)
df2['市值'] = df2['市值'].astype(float)#.astype(int)
round(df2['市值'].sum(), 0)

print((163583248000 - 163580362440) / 163580362440)

163580362440.0

1.7640014711780585e-05